<a href="https://colab.research.google.com/github/karan-mudaliar/Structured-Abstracts/blob/dev/notebooks/copali_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
loader = PyMuPDFLoader("attention.pdf")  
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  
    chunk_overlap=20  
)

chunks = text_splitter.split_documents(documents)

In [ ]:
model_name = "jinaai/jina-embeddings-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
def embed_text(text):
    # Tokenize input text and move to model’s device
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)  # Mean pooling of last hidden state
    return embeddings.squeeze().cpu().numpy()  # Convert to numpy for easier handling

# Generate embeddings for each chunk
chunk_embeddings = [embed_text(chunk.page_content) for chunk in chunks]

# Check the embedding shape
print("Embedding shape for each chunk:", chunk_embeddings[0].shape)